# Day 11
practicing with OOP instead of hacky

## GENERIC SETUP

In [1]:
# General imports
import pytest
import ipytest
import time
import functools

# Setup ipytest
ipytest.autoconfig()

# Setup nb_black
%load_ext nb_black

# Decorator to time solutions
def timer(func):
    """
    Wrapper function.
    Print the runtime of the decorated function.
    """

    @functools.wraps(func)
    def wrapper_timer(*args, **kwargs):
        start_time = time.perf_counter()  # 1
        value = func(*args, **kwargs)
        end_time = time.perf_counter()  # 2
        run_time = end_time - start_time  # 3
        print(f"Finished {func.__name__!r} in {run_time:.4f} secs")
        return value

    return wrapper_timer

<IPython.core.display.Javascript object>

## SOLUTION SETUP

#### I/O functions

In [2]:
# What day do we solve? Used to identify the input datafile, integer value
DAY = 11


def get_input():
    with open(f"../data/{DAY}.txt", "r") as f:
        return split_input(f.read())


def split_input(input_raw):
    """Strip trailing newline, then split on newline"""
    return [line.strip() for line in input_raw.rstrip().split("\n")]

<IPython.core.display.Javascript object>

#### Pytest input data

In [3]:
# Sample input
STR_SAMPLE = """\
L.LL.LL.LL
LLLLLLL.LL
L.L.L..L..
LLLL.LL.LL
L.LL.LL.LL
L.LLLLL.LL
..L.L.....
LLLLLLLLLL
L.LLLLLL.L
L.LLLLL.LL
"""


@pytest.fixture
def dummy_input():
    return STR_SAMPLE

<IPython.core.display.Javascript object>

## Solution A

In [4]:
class Position:
    """
    Class represents a single position in the waiting area.
    Can be floor, occupied or empty.
    """

    status_dict = {"#": True, "L": False, ".": False}

    def __init__(self, status_code):
        """Constructor. Requires a status code string as input"""
        self.status_code = status_code
        self.occupied = status_code == "#"
        self.seat = status_code in ("#", "L")

    def flip_seat(self):
        if self.occupied:
            self.occupied = False
            self.status_code = "L"
        else:
            self.occupied = True
            self.status_code = "#"

    # Auxiliary functions for operator/print functionality

    def __add__(self, other):
        """
        Override default add operator (+) functionality
        to count occupied positions
        """
        if isinstance(other, Position):
            return int(self.occupied) + int(other.occupied)
        else:
            return int(self.occupied) + int(other)

    def __radd__(self, other):
        """
        Necessary if summing over Seats in a list, base case tries to add self to 0.
        http://www.marinamele.com/2014/04/modifying-add-method-of-python-class.html
        """
        if other == 0:
            return self
        else:
            return self.__add__(other)

    def __repr__(self):
        return self.status_code

<IPython.core.display.Javascript object>

In [19]:
class WaitingArea:
    """
    Class represents a waiting area, consisting of a floorplan (grid)
    of seats (#/L) and empty spaces (.).
    A seat can be occupied (#) or unoccupied (L).

    Provides functionality to load a grid from an input string, and can support
    calculations on the grid by providing numbers of seats or lists of seats.
    
    Waiting Area comparisons can be done by representing the seating allocation as string,
    using the get_grid_as_string() function.
    """

    def __init__(self):
        """constructor, initialisation of local variables"""
        self.grid = None
        self.grid_height = 0
        self.grid_width = 0
        self.visible_seats_dict = {}  # PART B

    def load_string(self, input_lines):
        """Generates the seating grid from input string"""
        # Get the dimensions of the grid from input
        height = len(input_lines)
        width = len(input_lines[0])

        # Set the dimensions for the grid, including bounding dummies
        self.height = height
        self.width = width
        self.grid_height = height + 2
        self.grid_width = width + 2

        # Initialize the grid height
        new_grid = [None] * self.grid_height
        # Fill the first and last row with dummies
        new_grid[0] = [Position(".") for _ in range(self.grid_width)]
        new_grid[-1] = [Position(".") for _ in range(self.grid_width)]

        # fill the other rows
        for ix, line in enumerate(input_lines):
            ext_line = "." + line + "."
            new_grid[ix + 1] = [Position(input) for input in ext_line]

        # Store the grid
        self.grid = new_grid

        # Store input for unknown reasons
        self.source_string = input_lines

        print("[WaitingArea] Grid created from input")

        # Return self
        return self

    def get_surrounding(self, seat_row, seat_col):
        """
        Returns a list of the surrounding positions, excl. itself.
        Input coordinates do NOT take the dummy rows/cols into account.
        """
        row_ixs = range(seat_row, seat_row + 3)
        col_ixs = range(seat_col, seat_col + 3)

        surrounding_seats = [
            self.grid[row][col]
            for row in row_ixs
            for col in col_ixs
            if not (row == seat_row + 1 and col == seat_col + 1)
        ]
        return surrounding_seats

    def count_surrounding(self, seat_row, seat_col):
        """
        Count the occupied seats surrounding a position.
        Input arguments do NOT take into account the dummy rows.
        """
        surrounding_seats = self.get_surrounding(seat_row, seat_col)
        num_occupied = sum(surrounding_seats)
        return num_occupied

    def get_grid_as_string(self):
        """Generates a single string representing the seating configuration"""
        return "".join(
            [
                pos.status_code
                for row in range(self.grid_height)
                for pos in self.grid[row]
            ]
        )

    # Part B
    def get_visible_list(self, seat_row, seat_col):
        """
        Returns a list of coordinates of visible seats
        for a specific position in the grid.

        Input row/col take dummy rows into account (1-indexed).
        """
        # Determine the directions we can look at
        # direction = (delta row, delta col)
        directions = [
            (-1, -1),
            (-1, 0),
            (-1, 1),
            (0, -1),
            (0, 1),
            (1, -1),
            (1, 0),
            (1, 1),
        ]

        # Initialise list of visible positions
        visible_positions = []
        for (drow, dcol) in directions:
            row, col = (seat_row + drow, seat_col + dcol)
            # While we haven't found a visible seat and are not out of bounds,
            # Keep searching in the direction. If a seat found, add to visible-list
            while (1 <= row <= self.grid_height - 1) and (
                1 <= col <= self.grid_width - 1
            ):
                if self.grid[row][col].seat:
                    visible_positions.append((row, col))
                    break
                row, col = (row + drow, col + dcol)

        # Now visible_positions is populated with (row, col) coordinates
        return visible_positions

    def populate_visible_seats(self):
        """
        Pre-populate the visible_seats list of seats for each seat
        """
        for row in range(1, self.height + 1):
            for col in range(1, self.width + 1):
                if self.grid[row][col].seat:
                    self.visible_seats_dict[(row, col)] = self.get_visible_list(
                        row, col
                    )
        print("[WaitingArea] Visible seats pre-populated")
        return self

    # Auxiliary functions for the waiting area
    def __repr__(self):
        return str(self.grid)

<IPython.core.display.Javascript object>

In [6]:
import copy


def calculate_seating_round_A(waiting_area):
    """
    Given a waiting area, calculate a new seating allocation
    according to the problem A model.
    If a seat is empty (L) and there are no occupied seats adjacent to it,
        the seat becomes occupied.
    If a seat is occupied (#) and four or more seats adjacent to it are also occupied,
        the seat becomes empty.
    Otherwise,
        the seat's state does not change.
    """
    # Make our changes to new_grid
    new_grid = copy.deepcopy(waiting_area.grid)

    def determine_flip(waiting_area, new_grid, row, col):
        # Get the seat and number of surrounding occupied seats
        num_surr = waiting_area.count_surrounding(row, col)
        seat = waiting_area.grid[row + 1][col + 1]

        # Determine new state in new_grid
        if seat.occupied and num_surr >= 4:
            new_grid[row + 1][col + 1].flip_seat()
        elif not seat.occupied and num_surr == 0:
            new_grid[row + 1][col + 1].flip_seat()

    # Loop over the seats in the waiting area
    [
        determine_flip(waiting_area, new_grid, row, col)
        for row in range(0, waiting_area.height)
        for col in range(0, waiting_area.width)
        if waiting_area.grid[row + 1][col + 1].seat
    ]

    # new_grid now contains the new seating arrangement
    # Update the waiting area's grid to the new grid
    waiting_area.grid = new_grid

    return waiting_area


@timer
def solve_A(lines):
    """
    Initialise a waiting area from input,
    loop until equilibrium according to A,
    count #'s
    """
    # Initialize waiting area and loop variables
    waiting_area = WaitingArea().load_string(lines)
    new_seating = "INITIALISATION"
    old_seating = ""
    seat_round = 0

    # Repeat until equilibrium is found

    while old_seating != new_seating:
        seat_round += 1
        old_seating = new_seating
        calculate_seating_round_A(waiting_area)
        new_seating = waiting_area.get_grid_as_string()

    # Return the number of # in new_seating
    print(f"Finished with {seat_round} rounds.")
    return new_seating.count("#")

<IPython.core.display.Javascript object>

#### Tests

In [ ]:
%%run_pytest[clean] -qq

def test_A(dummy_input):
    assert solve_A(split_input(dummy_input)) == 37

#### OUTPUT

In [ ]:
solve_A(get_input())

## Solution B

In [22]:
def calculate_seating_round_B(waiting_area):
    """
    Given a waiting area, calculate a new seating allocation
    according to the problem B model.
    If a seat is empty (L) and there are no occupied seats visible to it,
        the seat becomes occupied.
    If a seat is occupied (#) and five (5) or more seats visible to it are also occupied,
        the seat becomes empty.
    Otherwise,
        the seat's state does not change.
    """
    # Make our changes to new_grid
    new_grid = copy.deepcopy(waiting_area.grid)

    def determine_flip(area, new_grid, row, col):
        # Get the seat and number of surrounding occupied seats
        seat = area.grid[row + 1][col + 1]
        visible_occupied = sum(
            [
                area.grid[r][c].occupied
                for r, c in area.visible_seats_dict[(row + 1, col + 1)]
            ]
        )

        # Determine new state in new_grid
        if seat.occupied and visible_occupied >= 5:
            new_grid[row + 1][col + 1].flip_seat()
        elif not seat.occupied and visible_occupied == 0:
            new_grid[row + 1][col + 1].flip_seat()

    # Loop over the seats in the waiting area
    [
        determine_flip(waiting_area, new_grid, row, col)
        for row in range(0, waiting_area.height)
        for col in range(0, waiting_area.width)
        if waiting_area.grid[row + 1][col + 1].seat
    ]

    # new_grid now contains the new seating arrangement
    # Update the waiting area's grid to the new grid
    waiting_area.grid = new_grid

    return waiting_area


@timer
def solve_B(lines):
    """
    Initialise a waiting area from input,
    loop until equilibrium according to B,
    count #'s
    """
    # Initialize waiting area and loop variables
    waiting_area = WaitingArea().load_string(lines)

    # Initialize for each seat the list of visible seats
    waiting_area.populate_visible_seats()

    # Initialize the variables for the seating loop
    new_seating = "INITIALISATION"
    old_seating = ""
    seat_round = 0

    # Repeat until equilibrium is found
    while old_seating != new_seating:
        seat_round += 1
        old_seating = new_seating
        calculate_seating_round_B(waiting_area)
        new_seating = waiting_area.get_grid_as_string()

    # Return the number of # in new_seating
    print(f"Finished with {seat_round} rounds.")
    return new_seating.count("#")

<IPython.core.display.Javascript object>

#### Tests

In [17]:
%%run_pytest[clean] -qq

def test_B(dummy_input):
    assert solve_B(split_input(dummy_input)) == 26

<IPython.core.display.Javascript object>

.                                                                                                             [100%]


<IPython.core.display.Javascript object>

#### OUTPUT

In [23]:
solve_B(get_input())

[WaitingArea] Grid created from input
[WaitingArea] Visible seats pre-populated
Finished with 91 rounds.
Finished 'solve_B' in 10.0660 secs


2285

<IPython.core.display.Javascript object>

# DEBUG B

In [20]:
testwa = WaitingArea().load_string(split_input(STR_SAMPLE)).populate_visible_seats()
testwa.visible_seats_dict

[WaitingArea] Grid created from input
[WaitingArea] Visible seats pre-populated


{(1, 1): [(1, 3), (2, 1), (2, 2)],
 (1, 3): [(1, 1), (1, 4), (2, 2), (2, 3), (2, 4)],
 (1, 4): [(1, 3), (1, 6), (2, 3), (2, 4), (2, 5)],
 (1, 6): [(1, 4), (1, 7), (2, 5), (2, 6), (2, 7)],
 (1, 7): [(1, 6), (1, 9), (2, 6), (2, 7), (4, 10)],
 (1, 9): [(1, 7), (1, 10), (4, 6), (2, 9), (2, 10)],
 (1, 10): [(1, 9), (2, 9), (2, 10)],
 (2, 1): [(1, 1), (2, 2), (3, 1), (4, 3)],
 (2, 2): [(1, 1), (1, 3), (2, 1), (2, 3), (3, 1), (4, 2), (3, 3)],
 (2, 3): [(1, 3), (1, 4), (2, 2), (2, 4), (4, 1), (3, 3), (5, 6)],
 (2, 4): [(1, 3), (1, 4), (2, 3), (2, 5), (3, 3), (4, 4), (3, 5)],
 (2, 5): [(1, 4), (1, 6), (2, 4), (2, 6), (4, 3), (3, 5), (4, 7)],
 (2, 6): [(1, 6), (1, 7), (2, 5), (2, 7), (3, 5), (4, 6), (5, 9)],
 (2, 7): [(1, 6), (1, 7), (2, 6), (2, 9), (5, 4), (4, 7), (3, 8)],
 (2, 9): [(1, 9), (1, 10), (2, 7), (2, 10), (3, 8), (4, 9)],
 (2, 10): [(1, 9), (1, 10), (2, 9), (5, 7), (4, 10)],
 (3, 1): [(2, 1), (2, 2), (3, 3), (4, 1), (4, 2)],
 (3, 3): [(2, 2), (2, 3), (2, 4), (3, 1), (3, 5), (4, 2), (

<IPython.core.display.Javascript object>